In [1]:
import re
import pandas as pd
from Bio import Entrez
from tqdm import tqdm
import time

In [2]:
# 设置 Entrez API 的邮箱地址
Entrez.email = "xiyangspace@foxmail.com"

In [3]:
# 读取输入文件
input_file = "/public/data_2/lxy/alto/project/brucella_wgs_Yunnan_2024/pipelines/denovo_ssembly/source/assembly_genbank/ncbi_assembly_brucella_melitensis.tsv"
output_file = "/public/data_2/lxy/alto/project/brucella_wgs_Yunnan_2024/pipelines/denovo_ssembly/source/assembly_genbank/ncbi_assembly_brucella_melitensis_metadata.tsv"

# 读取文件到 DataFrame
data = pd.read_csv(input_file, sep = "\t")
data = data[data["Assembly Accession"].str.startswith("GCA_")]

# 确保第 30 列为 BioSample ID
biosample_column_index = 29  # 第 30 列的索引为 29
data["collection_date"] = ""  # 添加新的列
data["geographic_location"] = ""
data = data.reset_index(drop = True)

In [4]:
# 遍历每行 BioSample ID，查询信息
# for index, row in tqdm(data.iterrows(), total = data.shape[0], desc = "Processing BioSample IDs"):
#     biosample_id = row.iloc[biosample_column_index]
#     try:
#         # 通过 Entrez 查询 BioSample 信息
#         handle = Entrez.efetch(db = "biosample", id = biosample_id, retmode = "xml")
#         biosample_data = handle.read().decode("utf-8")  # 解码为字符串
#         handle.close()

#         # 提取 collection date 和 geographic location
#         collection_date = ""
#         geographic_location = ""

#         if "collection date" in biosample_data:
#             collection_date = biosample_data.split("collection date")[1].split(">")[1].split("<")[0].strip()
#         else: 
#             collection_date = None

#         if "geographic location" in biosample_data:
#             geographic_location = biosample_data.split("geographic location")[1].split(">")[1].split("<")[0].strip()
#         else: 
#             geographic_location = None

#         # 更新表格
#         data.at[index, "collection_date"] = collection_date
#         data.at[index, "geographic_location"] = geographic_location

#     except Exception as e:
#         print(f"Error processing BioSample ID {biosample_id}: {e}")

In [5]:
# # 遍历每行 BioSample ID，查询信息
# for index, row in tqdm(data.iterrows(), total = data.shape[0], desc = "Processing BioSample IDs"):
#     biosample_id = row.iloc[biosample_column_index]
    
#     start_time = time.time() # 记录开始时间
    
#     # 通过 Entrez 查询 BioSample 信息
#     handle = Entrez.efetch(db = "biosample", id = biosample_id, retmode = "xml")
#     biosample_data = handle.read().decode("utf-8")  # 解码为字符串
#     handle.close()

#     # 提取 collection date 和 geographic location
#     collection_date = ""
#     geographic_location = ""

#     if "collection date" in biosample_data:
#         # 取 -1 而非 1，防止
#         # harmonized_name="collection_date" display_name="collection date">1962</Attribute>
#         collection_date = biosample_data.split("collection date")[-1].split(">")[1].split("<")[0].strip()
#     else: 
#         collection_date = None

#     if "geographic location" in biosample_data:
#         geographic_location = biosample_data.split("geographic location")[-1].split(">")[1].split("<")[0].strip()
#     else: 
#         geographic_location = None

#     # 更新表格
#     data.at[index, "collection_date"] = collection_date
#     data.at[index, "geographic_location"] = geographic_location
    
#     elapsed_time = time.time() - start_time  # 计算执行时间
    
#     # 如果执行时间不足 0.5 秒，则 sleep 至满足时间
#     if elapsed_time < 0.5:
#         time.sleep(min_time - elapsed_time)

In [6]:
# 遍历每行 BioSample ID，查询信息
for index, row in tqdm(data.iterrows(), total=data.shape[0], desc="Processing BioSample IDs"):
    biosample_id = row.iloc[biosample_column_index]
    collection_date = ""
    geographic_location = ""

    while True:  # 无限重试，直到成功
        try:
            start_time = time.time()  # 记录开始时间

            # 通过 Entrez 查询 BioSample 信息
            handle = Entrez.efetch(db="biosample", id=biosample_id, retmode="xml")
            biosample_data = handle.read().decode("utf-8")  # 解码为字符串
            handle.close()

            # 提取 collection date 和 geographic location
            if "collection date" in biosample_data:
                collection_date = biosample_data.split("collection date")[-1].split(">")[1].split("<")[0].strip()
            else: 
                collection_date = None

            if "geographic location" in biosample_data:
                geographic_location = biosample_data.split("geographic location")[-1].split(">")[1].split("<")[0].strip()
            else: 
                geographic_location = None

            # 更新表格
            data.at[index, "collection_date"] = collection_date
            data.at[index, "geographic_location"] = geographic_location

            elapsed_time = time.time() - start_time  # 计算执行时间

            # 如果执行时间不足 0.5 秒，则 sleep 至满足时间
            if elapsed_time < 0.5:
                time.sleep(0.5 - elapsed_time)

            break  # 成功后退出循环

        except Exception as e:
            print(f"Error encountered for BioSample ID {biosample_id}: {e}")  # 打印错误信息和 BioSample ID
            if "HTTP Error 400" in str(e):
                time.sleep(1)  # 如果捕获到 400 错误，延时 1 秒后重试
            else:
                raise  # 如果是其他错误，则重新抛出

Processing BioSample IDs:  52%|███████▊       | 389/747 [06:18<05:26,  1.10it/s]

Error encountered for BioSample ID SAMN40893475: HTTP Error 400: Bad Request


Processing BioSample IDs:  71%|██████████▋    | 534/747 [08:39<03:07,  1.14it/s]

Error encountered for BioSample ID SAMN03381084: HTTP Error 400: Bad Request


Processing BioSample IDs: 100%|███████████████| 747/747 [12:05<00:00,  1.03it/s]


In [7]:
data

,Assembly Accession,Assembly Name,Organism Name,Organism Taxonomic ID,ANI Check status,Organism Infraspecific Names Breed,Organism Infraspecific Names Strain,Organism Infraspecific Names Cultivar,Organism Infraspecific Names Ecotype,Organism Infraspecific Names Isolate,...,Assembly BioSample Accession,Annotation Count Gene Total,Annotation Count Gene Protein-coding,Annotation Count Gene Pseudogene,Type Material Display Text,CheckM marker set,CheckM completeness,CheckM contamination,collection_date,geographic_location
0,GCA_000007125.1,ASM712v1,Brucella melitensis bv. 1 str. 16M,224914,OK,NaN,16M,NaN,NaN,NaN,...,SAMN02603416,3264.0,3198.0,NaN,assembly from type material,Brucella melitensis,97.21,0.58,None,None
1,GCA_000740415.1,ASM74041v1,Brucella melitensis bv. 1 str. 16M,224914,OK,NaN,16M,NaN,NaN,NaN,...,SAMN02795970,3188.0,3110.0,15.0,assembly from type material,Brucella melitensis,98.25,0.58,unknown,unknown
2,GCA_003516085.1,ASM351608v1,Brucella melitensis,29459,OK,NaN,CIT43,NaN,NaN,NaN,...,SAMN08322135,3362.0,3128.0,167.0,NaN,Brucella melitensis,98.34,0.58,2015-03-01,"China: Kezuo Central Banner of Tongliao City, ..."
3,GCA_002263015.1,ASM226301v1,Brucella melitensis,29459,OK,NaN,BL,NaN,NaN,NaN,...,SAMN07508058,3378.0,3146.0,164.0,NaN,Brucella melitensis,98.37,0.58,2012,China: Xinjiang
4,GCA_036320855.1,ASM3632085v1,Brucella melitensis,29459,OK,NaN,B-HB-9,NaN,NaN,NaN,...,SAMN39517040,NaN,NaN,NaN,NaN,Brucella melitensis,97.53,0.64,2020-05-01,China: Hebei
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,GCA_947241995.1,B124,Brucella melitensis,29459,OK,NaN,NaN,NaN,NaN,B124,...,SAMEA111525744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
743,GCA_947242715.1,B153,Brucella melitensis,29459,OK,NaN,NaN,NaN,NaN,B153,...,SAMEA111525774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
744,GCA_947242335.1,B115,Brucella melitensis B115,1247411,OK,NaN,B115,NaN,NaN,NaN,...,SAMEA111525734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
745,GCA_000444515.1,Denovo assembly,Brucella melitensis ADMAS-G1,1351836,OK,NaN,ADMAS-G1,NaN,NaN,NaN,...,SAMN02470286,2797.0,2058.0,679.0,NaN,NaN,NaN,NaN,2013-03,India


In [8]:
data[data["collection_date"].isna() & data["geographic_location"].isna()].shape[0]

271

In [8]:
# 保存结果到新文件
data.to_csv(output_file, sep = "\t", index = False)
print(f"Results saved to {output_file}")

Results saved to /public/data_2/lxy/alto/project/brucella_wgs_Yunnan_2024/pipelines/denovo_ssembly/source/assembly_genbank/ncbi_assembly_brucella_melitensis_metadata.tsv
